In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'gpu')

In [2]:


sys.path.append("../..")

from lib import utils as U
from lib.ehr.dataset import load_dataset
from lib.ehr.inpatient_interface import Inpatients

In [3]:
import logging
logging.root.level = logging.DEBUG

In [4]:
from lib.ehr.coding_scheme import MIMIC4Procedures, MIMIC4ProcedureGroups
from lib.ehr.coding_scheme import MIMIC4Input, MIMIC4InputGroups

cproc = MIMIC4Procedures()
cproc_g = MIMIC4ProcedureGroups()
cinp = MIMIC4Input()
cinp_g = MIMIC4InputGroups()

In [5]:
cinp_g.groups


{'Abciximab': {'Abciximab (Reopro)'},
 'Acetaminophen-IV': {'Acetaminophen-IV'},
 'Acetylcysteine': {'Acetylcysteine'},
 'Adenosine': {'Adenosine'},
 'Alteplase (TPA)': {'Alteplase (TPA)'},
 'Ambisome': {'Ambisome'},
 'Amikacin': {'Amikacin'},
 'Amino Acids': {'Amino Acids'},
 'Aminocaproic acid (Amicar)': {'Aminocaproic acid (Amicar)'},
 'Aminophylline': {'Aminophylline'},
 'Amiodarone': {'Amiodarone', 'Amiodarone 450/250', 'Amiodarone 600/500'},
 'Ampicillin': {'Ampicillin'},
 'Ampicillin/Sulbactam (Unasyn)': {'Ampicillin/Sulbactam (Unasyn)'},
 'Angiotensin II (Giapreza)': {'Angiotensin II (Giapreza)'},
 'Argatroban': {'Argatroban'},
 'Atovaquone': {'Atovaquone'},
 'Atropine': {'Atropine'},
 'Azithromycin': {'Azithromycin', 'Erythromycin'},
 'Aztreonam': {'Aztreonam'},
 'Bactrim (SMX/TMP)': {'Bactrim (SMX/TMP)'},
 'Beneprotein': {'Beneprotein', 'Beneprotein.'},
 'Bivalirudin (Angiomax)': {'Bivalirudin (Angiomax)',
  'Bivalirudin (Angiomax) (Impella)'},
 'Boost Glucose Control': {'Boo

In [6]:
# Assign the folder of the dataset to `DATA_FILE`.
import dask

HOME = os.environ.get('HOME')
DATA_DIR = f'{HOME}/GP/ehr-data'
SOURCE_DIR = os.path.abspath("..")

with U.modified_environ(DATA_DIR=DATA_DIR), dask.config.set(scheduler='processes'):
    m4inpatient_dataset = load_dataset('M4ICU')
   

DEBUG:fsspec.local:open file: /home/asem/GP/ehr-data/mimic4icu-cohort/adm_df.csv
DEBUG:fsspec.local:open file: /home/asem/GP/ehr-data/mimic4icu-cohort/dx_df.csv
DEBUG:fsspec.local:open file: /home/asem/GP/ehr-data/mimic4icu-cohort/static_df.csv
DEBUG:fsspec.local:open file: /home/asem/GP/ehr-data/mimic4icu-cohort/obs_df.csv
DEBUG:fsspec.local:open file: /home/asem/GP/ehr-data/mimic4icu-cohort/int_input.csv
DEBUG:fsspec.local:open file: /home/asem/GP/ehr-data/mimic4icu-cohort/int_proc.csv
DEBUG:root:[DONE] Matching admission_id
DEBUG:root:Time casting..
DEBUG:root:[DONE] Time casting..
DEBUG:root:Constructing dx_icd9 (<class 'lib.ehr.coding_scheme.DxICD9'>) scheme
DEBUG:root:Constructing dx_icd10 (<class 'lib.ehr.coding_scheme.DxICD10'>) scheme
DEBUG:root:Constructing dx_icd9 (<class 'lib.ehr.coding_scheme.DxICD9'>) scheme
DEBUG:root:Constructing dx_icd9 (<class 'lib.ehr.coding_scheme.DxICD9'>) scheme
DEBUG:root:Constructing dx_icd9 (<class 'lib.ehr.coding_scheme.DxICD9'>) scheme
DEBUG:

In [7]:
splits = m4inpatient_dataset.random_splits([0.1, 0.7], 42, 'subjects')



In [8]:
preprocessing = m4inpatient_dataset.fit_preprocessing(splits[0])

In [9]:

m4inpatient_dataset.apply_preprocessing(preprocessing)

DEBUG:root:Removed 2431596 (0.024) outliers from obs


In [10]:
# from concurrent.futures import ThreadPoolExecutor
# with dask.config.set(pool=ThreadPoolExecutor(12)):
with dask.config.set(scheduler='processes', num_workers=12):
    m4inpatients = Inpatients(m4inpatient_dataset, splits[0][:10], num_workers=12)

DEBUG:root:Loading subjects..
/home/asem/GP/ICENODE/notebooks/mimic_icu/../../lib/ehr/dataset.py:970: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  dob = anchor_date + anchor_age
DEBUG:root:Constructing mimic4_eth32 (<class 'lib.ehr.coding_scheme.MIMIC4Eth32'>) scheme
DEBUG:root:Constructing mimic4_eth5 (<class 'lib.ehr.coding_scheme.MIMIC4Eth5'>) scheme
DEBUG:root:Extracting dx codes...
DEBUG:root:Constructing dx_icd10 (<class 'lib.ehr.coding_scheme.DxICD10'>) scheme
DEBUG:root:Constructing dx_icd9 (<class 'lib.ehr.coding_scheme.DxICD9'>) scheme
                            dx_icd10->dx_icd9 Unrecognised t_codes
                            (169):
                            ['041.41', '041.42', '041.43', '041.49', '173.00', '173.01', '173.02', '173.09', '173.10', '173.11', '173.12', '173.19', '173.20', '173.21', '173.22', '173.29', '173.30', '173.31', '173.32', '173.39']...
                            dx_icd10->dx_icd9 Unrecognised s_codes

In [11]:
m4inpatients.size_in_bytes() / 1024 ** 3

0.0022434135898947716

In [12]:
m4inpatients_jax = m4inpatients.to_jax_arrays(splits[0][:10])

DEBUG:jax._src.xla_bridge:Initializing backend 'cpu'
DEBUG:jax._src.xla_bridge:Backend 'cpu' initialized
DEBUG:jax._src.xla_bridge:Initializing backend 'cuda'
DEBUG:jax._src.xla_bridge:Backend 'cuda' initialized
DEBUG:jax._src.xla_bridge:Initializing backend 'rocm'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: CUDA Interpreter
DEBUG:jax._src.xla_bridge:Initializing backend 'tpu'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'


In [13]:
m4inpatients_jax.size_in_bytes() / 1024 ** 3

0.0022238409146666527

In [14]:
len(m4inpatients_jax.subjects)

10

In [15]:
m4inpatients_jax.n_admissions()

51

In [16]:
m4inpatients_jax.n_segments()

2549

In [17]:
m4inpatients_jax.n_obs_times()

1754

In [18]:
# import numpy as np
# import matplotlib.pyplot as plt

# a = m4inpatients_jax.obs_coocurrence_matrix
# a = np.array(a)
# plt.imshow(a, cmap='hot', interpolation='nearest')
# plt.show()

In [19]:
s = m4inpatients_jax.subjects[splits[0][6]].admissions[0].interventions.input_
s

InpatientInput(
  index=i32[100],
  rate=f16[100],
  starttime=f32[100],
  endtime=f32[100],
  size=318
)

In [20]:
m4inpatients_jax.interval_hours(splits[0][:10])

7167.583333333333

In [21]:
s

InpatientInput(
  index=i32[100],
  rate=f16[100],
  starttime=f32[100],
  endtime=f32[100],
  size=318
)

### التدريب على نموذج المعادلات التفاضلية الاعتيادية العصبية


In [22]:
from lib.ml.in_icenode import InICENODE, InICENODEDimensions
import jax.random as jrandom

DEBUG:matplotlib:matplotlib data path: /home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/asem/.config/matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:CACHEDIR=/home/asem/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/asem/.cache/matplotlib/fontlist-v330.json
/home/asem/GP/env/icenode-dev/lib/python3.9/site-packages/equinox/_ad.py:753: UserWarning: As of Equinox 0.10.7, `equinox.filter_custom_vjp.defvjp` is deprecated in favour of `.def_fwd` and `.def_bwd`. This new API supports symbolic zeros, which allow for more efficient autodifferentiation rules. In particular:
- the fwd and bwd functions take an extra `perturbed` argument, which     indicates which primals actually need a gradient. You can use this     to skip computing the gradient for any unperturbed value. (You can     also safely just ignore this if you wish.)
- `None` was pre

In [23]:
dims = InICENODEDimensions(state_m=15, 
                state_dx_e=10,
                state_obs_e=25,
                input_e=10,
                proc_e=10,
                demo_e=5,
                int_e=15)
key = jrandom.PRNGKey(0)

m = InICENODE(dims=dims, 
              scheme=m4inpatient_dataset.scheme,
              key=key)

DEBUG:jax._src.interpreters.pxla:Compiling _threefry_seed for with global shapes and types [ShapedArray(int32[])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _threefry_split for with global shapes and types [ShapedArray(uint32[2])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _unstack for with global shapes and types [ShapedArray(uint32[5,2])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _threefry_split for with global shapes and types [ShapedArray(uint32[2])]. Argument mapping: (GSPMDSharding({replicated}),).
DEB

DEBUG:jax._src.interpreters.pxla:Compiling squeeze for with global shapes and types [ShapedArray(uint32[1,2])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _unstack for with global shapes and types [ShapedArray(uint32[2,2])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling 

DEBUG:jax._src.interpreters.pxla:Compiling _threefry_split for with global shapes and types [ShapedArray(uint32[2])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _unstack for with global shapes and types [ShapedArray(uint32[1,2])]. Argument mapping: (GSPMDSharding({replicated}),).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Comp

DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_t

DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_t

DEBUG:jax._src.interpreters.pxla:Compiling dynamic_slice for with global shapes and types [ShapedArray(uint32[4,2]), ShapedArray(int32[]), ShapedArray(int32[])]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: 

DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling fn for with global shapes and types [ShapedArray(float32[], weak_type=True), ShapedArray(float32[50])]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type

DEBUG:jax._src.interpreters.pxla:Compiling _uniform for with global shapes and types [ShapedArray(key<fry>[]), ShapedArray(float32[], weak_type=True), ShapedArray(float32[], weak_type=True)]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]


In [25]:
res = m.batch_predict(m4inpatients_jax, splits[0][:10])

Embedding:   0%|                                                                            | 0/10 [00:00<?, ?subject/s]WARNING:jax._src.dispatch:Finished tracing + transforming matmul for pjit in 0.0007455348968505859 sec
DEBUG:jax._src.interpreters.pxla:Compiling embed_dx for with global shapes and types [ShapedArray(float32[50,17375]), ShapedArray(float32[50]), ShapedArray(float32[10,50]), ShapedArray(float32[10]), ShapedArray(bool[17375])]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_options: num_replicas=1 num_partitions=1 device_assignment=[[gpu(id=0)]]
DEBUG:jax._src.interpreters.pxla:Compiling demographic_vector for with global shapes and types [ShapedArray(float32[], weak_type=True), ShapedArray(bool[6])]. Argument mapping: (GSPMDSharding({replicated}), GSPMDSharding({replicated})).
DEBUG:jax._src.xla_bridge:get_compile_

NonConcreteBooleanIndexError: Array boolean indices must be concrete; got ShapedArray(bool[200])

See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.NonConcreteBooleanIndexError

In [ ]:
m4inpatients_jax.subjects[splits[0][0]].admissions[0].admission_dates